This lecture might seem a bit backwards to some - you've already learned how to manipulate dataframes
loading data from structured formats such as CSVs. But there are many more common data formats that pandas
can help you with, and I think it's important to have a bit more of a comprehensive look at data storage, so
here we're going to expand our investigation to other data formats.

In [ ]:
# Let's import the pandas library 
import pandas as pd

## CSV & Type Inference 



In [ ]:
# One of the most common and easiest files to work with is a CSV file or comma-separated text file. Loading,
# reading and writing CVS files is a key skill to master for any data scientist.

# To review,a  CSV is a txt file where values or columns are separated by commas CSV is quick to read and is
# compatible with most platforms, you can even open a CSV file with the text editor built into Jupyter

# While CSV is very easy to load, real world data rarly contains clean data in these cases, there are a few
# different pandas functions that can help out with messy data here is an example of a CSV file I pulled from
# google containing data on national education.
df= pd.read_csv ('datasets/National-2018-20190321.csv')
print(df.head())

# We can summerize the counts of unique values for a specific collumn, so the most frequently-occurring
# element and how much they occur
df['Qualification'].value_counts()

In [ ]:
# When you load a csv file, python knows that the values will be separated by commas, but CSV files don’t
# contain any type information for the data, so Pandas infers the data types unless you specify the value type
# for columns with the dtype parameter. We can quickly see the datatype infered by pandas using the DataFrame
# .dtypes attribute
df.dtypes

In [ ]:
# Usually the first row in a CSV file contain the names of the columns for the data, and when it doesn't
# you can either specify columns yourself or let pandas insert an autoincrementing number instead. A handy
# option is to actually set the index of the DataFrame to be one of the columns as well, on load.
pd.read_csv('datasets/National-2018-20190321.csv',index_col=1).head()

In [ ]:
# You can also insert a multiindex by just providing a list, and you can use column names as infered from the
# first row.
pd.read_csv('datasets/National-2018-20190321.csv', index_col=['Year Level', 'Academic Year']).head()

In [ ]:
# Sometimes a formatting issue can occur preventing you from working with your data at all. While you should
# always ensure you are working with the data you want to, it is sometimes useful to skip troublesome rows
# for data exploration
pd.read_csv('datasets/National-2018-20190321.csv', skiprows=[1, 2, 3]).head()

In [ ]:
# Sometimes you are dealing with large CSVs and reading them in one operation is intractable. To help deal
# with this you can give pandas a "chunksize", then iterate over dataframes in a chunk by chunk manner
chunker = pd.read_csv('datasets/National-2018-20190321.csv', chunksize=13)
i=0
for piece in chunker:
    print("Chunk {} has {} rows".format(i, len(piece)))
    i=i+1

In [ ]:
# I use this when I have a fairly extensive data cleaning or processing routine that I want to run on large
# datafiles to both reduce overall memory usage as well as improve CPU utilization. Now, this isn't
# particularly important for this class, but as you progress as a data scientist there will come a time when
# you need to load and work with bigish data - data that you should be able to work with on a laptop but can't
# - and knowing that there are places to investigate further is important.

In [ ]:
# Something you should be aware with chunking, however, is that the pandas type inference works on a chunk-by-
# chunk size. So your types might end up being a bit unpredictictable. In the above example there is one cell
# in the "Academic Year" column which has a non-number. When we read in the whole dataframe this is read as
# a type of object. But when we read in by chunks we see this is only an object for one chunk.
for piece in pd.read_csv('datasets/National-2018-20190321.csv', chunksize=13):
    print(piece["Academic Year"].dtype)
    i=i+1

## Excel 

In [ ]:
# Perhaps even more ubiquitous than CSV files in the corporate world are Excel files. You'll find that much
# of the data created or given to you might be trapped in this format. Using the excellent xlrd library,
# pandas is able to read this almost as seamlessly as CSV files.

# An important difference from CSV files is that a single Excel workbook might contain many sheets, each
# analagous to a DataFrame. Lets load some data from a report card on Elementary Districts from 2016-17
# and here I'll just look at one sheet, the Primary Class Sizes
pd.read_excel('datasets/classsizes.xlsx',sheet_name='Primary Class Sizes').head()

In [ ]:
# Hrm. That table doesn't look so useful, let's take a look at what this looks like in Excel

![Excel](excel.png)

In [ ]:
# Ok, so this is a pretty typical data file you might be handed to be able to work with. We see a number
# of different rows being taken up with graphics and the like, colors being used for headers, and that there
# are at least two DataFrames on this page. To deal with this you're going to want to use a number of the
# different options in the read_excel() function.

# Excel numbers its rows starting at 1, but we don't, we start at 0. And Excel has columns starting with A,
# but with pandas we use numbers instead of letters, again starting at 0. Lets grab that top table into a df
df=pd.read_excel('datasets/classsizes.xlsx',sheet_name='Primary Class Sizes',
              header=7,skipfooter=19,index_col=0,dtype={'Average class size': str})
df.head()

## Pickle 

In [ ]:
# The easiest way to store data in binary format is using python's built-in pickle serialization. Now, this
# isn't a pandas library, this is built into the language. It's quick and easy to dump an object, like a
# DataFrame, to a file to use it later. There are lots of caveats to doing this and I wouldn't recommend it
# over some of the other formats we'll talk about, but if you're working mostly with Python developers you
# will undoubtedly need to use it.
#
# And, one of the most useful parts of pickle versus some of the other formats we have talked about is that
# datatypes are preserved. So you can do all of your cleaning and typing in one file, then share the object
# with others.

In [ ]:
# First lets import pickle 
import pickle

# Now we are going to create the pickle file we want to write to
with open('dataframe.pickle', 'wb') as f:
    # Now we just tell python to dump our dataframe from the previous example into this pickle file
    pickle.dump(df, f)

In [ ]:
# And when we want to read it in again we can do so easily
with open('dataframe.pickle', 'rb') as f:
    our_old_dataframe = pickle.load(f)
our_old_dataframe

In [ ]:
# Pickle files are a quick and easy way to store DataFrames, and I find that sometimes I use them when
# I need to save intermediate DataFrames, for instance when I'm working with large dataframes on a machine
# with a limited amount of memory, or when I need to distribute portions of a dataframe to different
# processes or machines.

## HDF5 

In [ ]:
# HDF5 is used when you're working with very large datasets, and it is especially useful for datasets that 
# wont fit into memory. "HDF" stands for hierarchical data format, and each HDF5 file can store multiple 
# datasets as well as supporting metadata (like column information). HDF5 supports on-the-fly compression 
# with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently 

# Let's bring in the numpy library
import numpy as np

# To use HDF5 we first create our data store as a file
store = pd.HDFStore('mydata.h5')

# Then we can insert into that store different DataFrames or even portions of frames
store['class_sizes']=df
store['class_size_2009']=df[2009]

store

In [ ]:
# You can see what keys are available in the HDFStore with .keys()
store.keys()

In [ ]:
# Objects contained in the HDF5 file can then be retrieved with the same dict-like API 
store['class_sizes'].head()

In [ ]:
# But where HDF5 really shines is when you save  an object of type "table"
store.put('a_new_class_size', df, format='table')
store.keys()

In [ ]:
# When you do this, you can retrieve portions of your DataFrame from disk without having to read the whole
# DataFrame into memory
store.select('a_new_class_size', where=['index=Prep'], columns=[2009,2010])

In [ ]:
# A deeper investigation of HDF5 is outside of the scope of this lecture, but I would encourage you to check
# out the online docs and play a bit with it, especially the novel heirarchical nature of the data. Where it
# is used most commonly is large scientific datasets which are collected once but read many times. A more
# common alternative, which we won't cover in this course due to it's size and complexity, are relational
# databases, and there are several ways of interacting with these data sources from within python and pandas.

## Pyarrow 

In [ ]:
# The last datafile format I'll talk about here is Pyarrow and Feather format. The Feather format is an
# open-source columnar storage format - so a DataFrame format - which intended to be a replacement for CSV
# files and is both faster to read and write as well as preserves metadata about columns.

# Let's do some time comparisons reading in the same dataframe from both CSV and Feather formats

In [ ]:
%%timeit -n 10
# First the CSV reading
df_csv=pd.read_csv("datasets/house_prices.csv")

In [ ]:
%%timeit -n 10
# Now the feather reading
df_feather=pd.read_feather("datasets/house_prices.feather")

In [ ]:
# So, we see that the feather reading is moderatly faster. Lets look at the dataframe types and compare
# them
df_csv=pd.read_csv("datasets/house_prices.csv")
print(df_csv.dtypes)
df_feather=pd.read_feather("datasets/house_prices.feather")
print(df_feather.dtypes)

In [ ]:
# One difference we can see is that the sale_date field is correctly set to datetime64 in the feather
# DataFrame, but was just interpreted as a generic object in the CSV dataframe. Now, we could probably further
# tweak some of these datatypes to increase speed a little bit if we wanted to, dropping the size of the
# integer and float values

In [ ]:
# But I don't want to give you the thought you should pivot and start using feather for everything. It's very
# much under active development, and there are some potential catches or defaults that you might not expect.
# For instance, I thought preserving a multi-index would work, but it doesn't
df_feather=df_feather.set_index(["state","city"])
df_feather.to_feather("house_prices.feather")

In [ ]:
# The default behavior for CSVs is different, and the to_csv() function writes the columns (though not as a
# multi-index, just as regular columns). Regardless, feather is a great format to watch evolve, and the 
# bindings across programming languages means there is potential for high interoperability and high
# performance of DataFrame data structures.

In this lecture we covered a range of datafile formats for DataFrames. By far the most common is the CSV,
but it's useful to know that there are other formats you might have to work with, and that pandas supports
them. Excel files are very common in large organizations, pickle files are quick to move between python
processes and capture all the nuance of the DataFrame object, and HDF5 and Feather files have their uses in
more specialized circumstances.